In [1]:
!pip install hypertools

In [2]:
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr
import sys
import os
sys.path.append(os.path.abspath('../'))
from timecorr.timecorr import timepoint_decoder
import matplotlib.pyplot as plt
from math import log

sliding_window_length = 51
block_length = 1
covariance_num = 1000
noise_level = [0.01,0.1,1,10,100]
repetitions=100
noise_num = len(noise_level)
time_range=time_len = block_length * covariance_num
activation_num = 6
subject_num = 5
variance = 1000
activations = np.random.normal(0,1,[noise_num, subject_num, activation_num, time_len])
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])


def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations_temp = np.random.normal(0,1,[activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map1,feature_map1.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map2 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map2,feature_map2.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len):
        cov_temp = (time_len-i)*0.5*(np.log(1+correlation1+1e-5) - np.log(1-correlation1+1e-5))/float(time_len)+i*0.5*(np.log(1+correlation2+1e-5) - np.log(1-correlation2+1e-5))/float(time_len)
        correlations[i] =  (np.exp(2*cov_temp) - 1)/(np.exp(2*cov_temp) + 1)
        activations_temp[:,i] = np.dot(cholesky(correlations[i]),activations_temp[:,i])
    for i in range(noise_num):
        activations[i]=np.tile(activations_temp,[subject_num,1,1] )+np.random.normal(0,noise_level[i],[subject_num, activation_num, time_len])
        
cholesky_ramp_correlation_data()
for i in range(noise_num):
    print("noise level "+ str(noise_level[i])+" results: "+str(timepoint_decoder(activations[i],nfolds=10)))

{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
noise level 0.01 results: {'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
noise level 0.1 results: {'ac

In [3]:
import numpy as np
from scipy.linalg import cholesky
from scipy.spatial.distance import squareform
from scipy.stats.stats import pearsonr
import sys
import os
sys.path.append(os.path.abspath('../'))
from timecorr.timecorr import timepoint_decoder
import matplotlib.pyplot as plt
from math import log

sliding_window_length = 51
block_length = 1
covariance_num = 1000
noise_level = [0]
repetitions=100
noise_num = len(noise_level)
time_range=time_len = block_length * covariance_num
activation_num = 6
subject_num = 5
variance = 1000
activations = np.random.normal(0,1,[noise_num, subject_num, activation_num, time_len])
correlations = np.zeros([covariance_num,activation_num,activation_num])
correlation1,correlation2 = np.zeros([activation_num,activation_num]), np.zeros([activation_num,activation_num])


def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def cholesky_ramp_correlation_data():
    global activations, correlations,correlation1,correlation2
    correlations = np.zeros([covariance_num,activation_num,activation_num])
    activations_temp = np.random.normal(0,1,[activation_num, time_len])
    
    while not is_pos_def(correlation1):
        feature_map1 = np.random.normal(0,1,[activation_num,activation_num])
        correlation1 = np.dot(feature_map1,feature_map1.T)
        correlation1 = correlation1/np.max(abs(correlation1))
    while not is_pos_def(correlation2):
        feature_map2 = np.random.normal(0,1,[activation_num,activation_num])
        correlation2 = np.dot(feature_map2,feature_map2.T)
        correlation2 = correlation2/np.max(abs(correlation2))
    for i in range(time_len):
        cov_temp = (time_len-i)*0.5*(np.log(1+correlation1+1e-5) - np.log(1-correlation1+1e-5))/float(time_len)+i*0.5*(np.log(1+correlation2+1e-5) - np.log(1-correlation2+1e-5))/float(time_len)
        correlations[i] =  (np.exp(2*cov_temp) - 1)/(np.exp(2*cov_temp) + 1)
        activations_temp[:,i] = np.dot(cholesky(correlations[i]),activations_temp[:,i])
    for i in range(noise_num):
        activations[i]=np.tile(activations_temp,[subject_num,1,1] )+np.random.normal(0,noise_level[i],[subject_num, activation_num, time_len])
        
cholesky_ramp_correlation_data()
print("Results: "+str(timepoint_decoder(activations[0],nfolds=10)))

/Users/ThousandSunny/Documents/Research/timecorr/timecorr/timecorr.py:27: RuntimeWarning: divide by zero encountered in log
  return cfun(x, var)
/Users/ThousandSunny/Documents/Research/timecorr/timecorr/timecorr.py:27: RuntimeWarning: invalid value encountered in log
  return cfun(x, var)
/Users/ThousandSunny/Documents/Research/timecorr/timecorr/timecorr.py:27: RuntimeWarning: invalid value encountered in divide
  return cfun(x, var)


{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
{'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
Results: {'accuracy': 1.0, 'rank': 1.0, 'error': 0.0}
